<a href="https://colab.research.google.com/github/GeanRaffo/robo-juros-abusivos/blob/main/M%C3%B3dulo_de_An%C3%A1lise_de_Juros_(revisajuros_py).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Módulo: revisajuros.py
import requests
from datetime import datetime
from math import pow

# Mapeamento de modalidades para códigos do BCB (Pessoa Física)
MODALIDADE_CODIGOS_PF = {
    "veiculos": 25464,
    "pessoal": 25469,
    "consignado_inss": 25467,
    "consignado_publico": 25468,
    "cartao_rotativo": 25354,
    "cheque_especial": 25471
}

def anualizar_taxa_mensal(taxa_mensal):
    """Converte uma taxa de juros mensal para uma taxa anual."""
    return (pow((1 + taxa_mensal / 100), 12) - 1) * 100

def buscar_taxa_bcb(codigo, data):
    """Busca a taxa de juros média mensal do BCB para uma data específica."""
    try:
        data_formatada = datetime.strptime(data, "%Y-%m-%d").strftime("%d/%m/%Y")
        url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo}/dados?formato=json&dataInicial={data_formatada}&dataFinal={data_formatada}"
        resposta = requests.get(url, timeout=5)
        if resposta.status_code == 200 and resposta.json():
            valor_str = resposta.json()[0]['valor']
            return float(valor_str)
        else:
            # Fallback: Se não encontrar para a data exata, tenta buscar os últimos 30 dias e pegar o mais recente
            data_inicial_fallback = (datetime.strptime(data, "%Y-%m-%d") - timedelta(days=30)).strftime("%d/%m/%Y")
            url_fallback = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo}/dados?formato=json&dataInicial={data_inicial_fallback}&dataFinal={data_formatada}"
            resposta_fallback = requests.get(url_fallback, timeout=5)
            if resposta_fallback.status_code == 200 and resposta_fallback.json():
                return float(resposta_fallback.json()[-1]['valor'])
    except Exception as e:
        print(f"Erro ao buscar taxa do BCB: {e}")
        return None
    return None

def analisar(dados_contrato):
    """
    Realiza a análise completa dos juros e retorna um dicionário com o relatório.

    Args:
        dados_contrato (dict): Dicionário com os dados do formulário.

    Returns:
        dict: Um dicionário contendo os resultados da análise.
    """
    taxa_contrato = dados_contrato.get('taxaContrato')
    periodo = dados_contrato.get('periodo')
    modalidade = dados_contrato.get('modalidade')
    data_contrato = dados_contrato.get('dataContrato')

    if periodo == 'mes':
        taxa_anual_contrato = anualizar_taxa_mensal(taxa_contrato)
    else:
        taxa_anual_contrato = taxa_contrato

    codigo_bcb = MODALIDADE_CODIGOS_PF.get(modalidade)
    if not codigo_bcb:
        return {"erro": "Modalidade de crédito inválida."}

    taxa_media_bcb_mensal = buscar_taxa_bcb(codigo_bcb, data_contrato)

    if taxa_media_bcb_mensal is None:
        return {"erro": f"Não foi possível obter a taxa média do BCB para a data e modalidade informadas."}

    taxa_media_bcb_anual = anualizar_taxa_mensal(taxa_media_bcb_mensal)

    diferenca = taxa_anual_contrato - taxa_media_bcb_anual
    percentual_acima = (diferenca / taxa_media_bcb_anual) * 100 if taxa_media_bcb_anual > 0 else float('inf')

    veredito_msg = ""
    veredito_status = "ok" # 'ok', 'atencao', 'alerta'

    if percentual_acima > 50:
        veredito_msg = "ALERTA: Indícios FORTES de juros abusivos foram encontrados."
        veredito_status = "alerta"
    elif percentual_acima > 25:
        veredito_msg = "ATENÇÃO: Indícios MODERADOS de juros abusivos. Recomenda-se análise aprofundada."
        veredito_status = "atencao"
    else:
        veredito_msg = "ANÁLISE: A taxa do contrato parece estar dentro dos limites da média de mercado."
        veredito_status = "ok"

    return {
        "dados_contrato": dados_contrato,
        "taxa_anual_contrato": f"{taxa_anual_contrato:.2f}%",
        "taxa_media_bcb_anual": f"{taxa_media_bcb_anual:.2f}%",
        "diferenca_nominal": f"{diferenca:.2f}%",
        "percentual_acima_media": f"{percentual_acima:.2f}%",
        "veredito_msg": veredito_msg,
        "veredito_status": veredito_status
    }